In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [2]:
F = pd.read_csv(mm3_url)
f = remove_empty_columns(F)
f = merge_multiple_academic_years(f)


/workspaces/academic-staff-workload/rsi_modules.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(how='all', axis=1, inplace=True)
/workspaces/academic-staff-workload/rsi_modules.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['academic_year'] = expand.values()


In [5]:
for p,ci in enumerate(f["รหัสวิชา"]):
    fdi = extract_non_empty_cells(f,ci)
    break

In [8]:
fdi[fdi['items']=="academic_year"]['data'].to_numpy()[0]

'[2568]-เทอม 1'

In [ ]:
def courses(f):
    cols  = f.keys()
    data = defaultdict(lambda:defaultdict(list))
    lec  = defaultdict(lambda:defaultdict(float))
    courses_year = defaultdict(list)
    # ดึงทีละวิชา
    for p,ci in enumerate(f["รหัสวิชา"]):
        fdi = extract_non_empty_cells(f,ci)
        # ดึงเทอมและปีการศึกษา
        yr_term = fdi[fdi['items']=="academic_year"]['data'].to_numpy()[0]
        # ดึงชั่วโมงบรรยาย
        hr_lecture = int(fdi[fdi['items']=="จำนวน หน่วยกิต (ทฤษฎี-ปฏิบัติ-เรียนรู้ด้วยตนเอง) [ทฤษฎี]"]['data'].to_numpy()[0])
        # ดึงผู้สอนหลัก
        fdii = fdi.loc[(fdi['data'] == 'ผู้สอนหลัก') | (fdi['data'] == 'ผู้สอนหลัก, ผู้ปฏิบัติ')].reset_index(drop=True)
        for i in range(1,17):
            J = [j for j in fdii['items'] if f'(คาบที่ {i})' in j]
            if len(J) > 0:
                data[i][ci] = hr_lecture/len(J)
                for jj in J:
                    lec[ci][jj.split('[')[1].split(']')[0]]+=hr_lecture/len(J)
            else:
                data[i][ci] = 0

        if 'รมคพ 705' in ci: lec = special_hr_705(lec,ci)
        courses_year[yr_term].append(lec[ci])

    return lec, courses_year

In [ ]:
lec, courses_year = courses(f)
courses_year = calculate_teaching_hours_per_year(courses_year)
pd.DataFrame(courses_year).fillna(0).to_csv("courses_year.csv", index=False)

In [ ]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
# df.to_csv('publication.csv')
# st.write(df)

In [ ]:
def classify_publication(publication):  

    NAMES = []
    
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n) 

    for t in publication.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)

    name_paper = defaultdict(list)
    for name in NAMES:
        papers = []
        for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],
                                                            publication['วารสารไทย'],
                                                            publication['วารสารนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับชาติ'])):
            if role in ['First Author', 'Corresponding Author']:
                papers+=[TCI+Q+conf_int+conf_nat]
        name_paper[name.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [','').replace(']','')] = papers

    categories = ['Q1','Q2','Q3','Q4','TCI1','TCI2','อื่นๆ']
    blanktable = pd.DataFrame(np.zeros((len(name_paper.keys()), len(categories)),dtype=int), columns=categories)
    blanktable['name']=name_paper.keys()


    for idx, (name, papers) in enumerate(name_paper.items()):
        for paper in papers:
            for cat in categories:
                if cat in paper:
                    blanktable.loc[idx, cat] += 1
    
    # Reorder columns with 'name' first
    blanktable = blanktable[['name'] + [col for col in blanktable.columns if col != 'name']]
    
    # blanktable.to_csv('publication_classified.csv')

    return blanktable

In [ ]:
def get_publications_by_author(publication):
    """
    Create a dictionary where:
    - Key: author name (First Author or Corresponding Author)
    - Value: list of tuples (year, title, publication_type)
    """
    
    # Find author columns
    NAMES = []
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n)
    
    # Rename journal/conference columns for easier processing
    pub_copy = publication.copy()
    for t in pub_copy.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: 
            pub_copy.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: 
            pub_copy.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)
    
    author_publications = defaultdict(list)
    
    # Get year and title columns
    year_col = 'ปีที่ตีพิมพ์ พศ. (คศ.)'
    title_col = 'ชื่อเรื่อง'
    
    # Process each row
    for idx, row in pub_copy.iterrows():
        year = row.get(year_col, '')
        title = row.get(title_col, '')
        
        # Determine publication type (Q or TCI category)
        pub_type = 'อื่นๆ'
        for category in ['Q1', 'Q2', 'Q3', 'Q4', 'TCI1', 'TCI2']:
            combined_text = (str(row.get('วารสารไทย', '')) + 
                           str(row.get('วารสารนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับชาติ', '')))
            if category in combined_text:
                pub_type = category
                break
        
        # Process each author
        for name_col in NAMES:
            role = row.get(name_col, '')
            if role in ['First Author', 'Corresponding Author','EIC']:
                if role == 'First Author': role = 'FA'
                elif role == 'Corresponding Author': role = 'CA'
                # Extract author name from column name
                author_name = name_col.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [', '').replace(']', '')
                # Add publication tuple (year, title, type)
                author_publications[author_name].append((year, pub_type, role, title))
    
    return dict(author_publications)


# Export author_pubs to CSV
def export_author_publications_to_csv(author_pubs, filename):
    """
    Export author publications dictionary to CSV file
    Format: Author, Year, Publication Type, Title
    """
    rows = []
    for author, publications in author_pubs.items():
        for year, pub_type, role, title in publications:
            rows.append({
                'Author': author,
                'Year': year,
                'Publication Type': pub_type,
                'Role': role,
                'Title': title
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Exported {len(rows)} publications to {filename}")
    return df

# Test the function
author_pubs = get_publications_by_author(publication)
df_export = export_author_publications_to_csv(author_pubs, 'author_publications.csv')


In [ ]:
author_pubs

In [ ]:

# Export to CSV
df_export.head()


In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

In [ ]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [ ]:
st.write(A)